# Training a Two-Layer Neural Network on Cifar-10 --

The tendril classification problem allowed us to use Neural Networks on a 2D toy dataset. In this notebook, we will work with an $n$-dimensional dataset of images, where $n$ is the total size (# pixels x # color-channels) of an image. We will be using the famed [cifar-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html), so that our  model can classify pictures of cars, planes, cats, dogs, frogs, and other items. There are 10 classes in total represented in this dataset. Each image has is an 32 pixels by 32 pixels RGB image of shape ``(3,32,32)``. Thus each image is a point or vector in $\mathbb{R}^{3072}$.

We will be training a two-layer neural network. Our loss function is the cross-entropy loss. The first two layers will use the ReLU activation function and the last layer will use softmax activation.


#### The Model in Full

\begin{equation}
D_1(x) = \operatorname{ReLU}(xW_{1} + b_{1})\\
D_2(x) = \operatorname{ReLU}(D_1(x) W_{2} + b_{3})\\
F(\{W\}, \{b\}; x) = \operatorname{softmax}(D_2(x) W_3+b_3)
\end{equation}


We will again be using the popular cross-entropy classification loss. Keep in mind that `mygrad`, and other auto-differentiation libraries, provide a convenient softmax_crossentropy function, which efficiently computes the softmax *and then* the corss-entropy. So take care to not invoke softmax twice, in  following the equations above.

In [ ]:
import matplotlib.pyplot as plt

import mygrad as mg
import numpy as np

%matplotlib notebook

In [ ]:
import datasets
datasets.download_cifar10()
x_train, y_train, x_test, y_test = datasets.load_cifar10()

print('Training data shape: ', x_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)
print(x_train.dtype)

Let's investigate what our data roughly looks like. Plotting some sample images from each class.

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(x_train[idx].transpose(1,2,0).astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

Flatten out x_train and x_test and use ``astype`` to convert your data to ``np.float32``. Your ``(3,32,32)`` image should now be ``(3072,)``. Additionally, find the mean image and standard deviation image of the training and test data. Then, zero-center your data by subtracting the mean image and normalize by dividing out by the standard deviation image.

In [ ]:
# Reshape the train and test data to be shape (N, 3072)  (N=50,000 for train, N=10,000 for test)
# STUDENT CODE HERE

# Using only the training data, compute the mean-image (a shape-(3072,) array)
# and the std-dev image (also shape-(3072,) array)
#
# That is, we are computing the mean 1st pixel over all images, the mean 2nd pixel,
# and so on, creating a shape-(3072,) array of these mean pixel values.
# STUDENT CODE HERE

# Using the *same* mean and std, do a pixel-wise normalization of the testing
# data: x_test = (x_test - mean) / std
# STUDENT CODE HERE

Now, let's construct our model using `MyNN` and define our [accuracy function](https://www.pythonlikeyoumeanit.com/Module3_IntroducingNumpy/Problems/ComputeAccuracy.html).

We can experiment with the sizes of our layers, but try:

- layer-1: size-100
- layer-2: size-50
- layer-3: size-? (hint: we don't get to pick this)
  - This final layer should not have any bias: `dense(..., bias=False)`
  - We will be using `softmax_crossentropy` for our loss, we don't need an activation function for this last layer.

Use the `he_normal` initialization for each layer.

In [ ]:
from mygrad.nnet.initializers.he_normal import he_normal
from mygrad.nnet.activations.relu import relu
from mygrad.nnet.losses import softmax_crossentropy

from mynn.optimizers.sgd import SGD
from mynn.layers.dense import dense


# Define your MyNN-`Model` class here. It should have:
# - an `__init__` method that initializes all of your layers
# - a `__call__` method that defines the model's "forward pass"
# - a `parameters` property that returns a tuple of all of your
#   model's learnable parameters (refer to the Tendrils-MyNN)
#   notebook for the syntax of defining a class-property)

class Model:
    def __init__(self, n1, n2, num_classes):
        """
        Initializes a model with two hidden layers of size `n1` and `n2`
        respectively.

        Parameters
        ----------
        n1 : int
            The number of neurons in the first hidden layer

        n2 : int
            The number of neurons in the second hidden layer

        num_classes : int
            The number of classes predicted by the model"""
        # STUDENT CODE HERE

Initialize your model and optimizer, using SGD from MyNN. Specify the parameters, learning rate and weight_decay for your
optimizer.

A learning rate of $0.1$ and a weight decay of $5\times10^{-4}$ is sensible

In [ ]:
# STUDENT CODE HERE

Now write code to train your model! Experiment with your learning rate and weight_decay.

In [ ]:
# Set `batch_size = 100`: the number of predictions that we will make in each training step

# STUDENT CODE HERE

# We will train for 10 epochs; you can change this if you'd like.
# You will likely want to train for much longer than this
for epoch_cnt in range(10):

    # Create the indices to index into each image of your training data
    # e.g. `array([0, 1, ..., 9999])`, and then shuffle those indices.
    # We will use this to draw random batches of data
    # STUDENT CODE HERE

    for batch_cnt in range(0, len(x_train) // batch_size):
        # Index into `x_train` to get your batch of M images.
        # Make sure that this is a randomly-sampled batch
        # STUDENT CODE HERE

        # compute the predictions for this batch by calling on model
        # STUDENT CODE HERE


        # compute the true (a.k.a desired) values for this batch:
        # STUDENT CODE HERE


        # compute the loss associated with our predictions(use softmax_cross_entropy)
        # STUDENT CODE HERE


        # back-propagate through your computational graph through your loss
        # STUDENT CODE HERE


        # execute gradient-descent by calling step() of optim
        # STUDENT CODE HERE


        # compute the accuracy between the prediction and the truth
        # STUDENT CODE HERE


        plotter.set_train_batch({"loss" : loss.item(),
                                 "accuracy" : acc},
                                 batch_size=batch_size)

    # After each epoch we will evaluate how well our model is performing
    # on data from cifar10 *that it has never "seen" before*. This is our
    # "test" data. The measured accuracy of our model here is our best
    # estimate for how our model will perform in the real world
    # (on 32x32 RGB images of things in this class)
    test_idxs = np.arange(len(x_test))

    for batch_cnt in range(0, len(x_test)//batch_size):
        batch_indices = test_idxs[batch_cnt*batch_size : (batch_cnt + 1)*batch_size]

        batch = x_test[batch_indices]
        truth = y_test[batch_indices]

        # We do not want to compute gradients here, so we use the
        # no_autodiff context manager to disable the ability to
        with mg.no_autodiff:
            # Get your model's predictions for this test-batch
            # and measure the test-accuracy for this test-batch
            # STUDENT CODE HERE

        # pass your test-accuracy here; we used the name `test_accuracy`
        plotter.set_test_batch({"accuracy" : test_accuracy}, batch_size=batch_size)
    plotter.set_test_epoch()

## Evaluating Your Results

How well is your model performing? Is there any discrepancy between how well it does on training data vs testing data?

Below, we provide code to randomly pick an image from the test set, plot it, and print your model's predicted label vs the true label. `datasets.load_cifar10.labels` returns a tuple of the label-names in correspondence with each truth-index.

Since we shifted and normalized our data, we have to re-load the data here, using different names for the arrays.

Note that we still need to pass your model the shifted/normalized test images. So the data you use to plot the image is different from the data that you pass to the model. Also note that your model expects a *batch* of images, not a single image. Thus we use a batch of size-1, which has shape-(1, 3072) - your model will produce a shape-(1, 10) tensor of predictions.

In [ ]:
_, _, img_test, label_test = datasets.load_cifar10()

In [ ]:
labels = datasets.load_cifar10.labels  # tuple of cifar-10 labels

index = np.random.randint(0, len(img_test))  # pick a random test-image index

true_label_index = label_test[index]
true_label = labels[true_label_index]

with mg.no_autodiff:
    prediction = model(x_test[index:index + 1])  # you must pass in a shape-(1, 3072) array
    predicted_label_index = np.argmax(prediction.data, axis=1).item()  # largest score indicates the prediction
    predicted_label = labels[predicted_label_index]


fig, ax = plt.subplots()

# matplotlib wants shape-(H, W, C) images, with unsigned 8bit pixel values
img = img_test[index].transpose(1,2,0).astype('uint8')

ax.imshow(img)
ax.set_title(f"Predicted: {predicted_label}\nTruth: {true_label}");

Can you understand some of the mistakes that your model is making? Perhaps it sees a white plane over water, and confuses it for a boat. Can *you* figure out what some of these images depict? Some are pretty hard to identify, given the low resolution.